In [1]:
import pandas as pd
import numpy as np
import json
import copy
from tqdm import tqdm 
from gensim.models import Word2Vec

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
def seed_everything(seed=7):
#     random.seed(seed)
#     os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything(seed=107)

In [3]:
train_root = './data/train_semi_final/'
test_root = './data/test/'
# ad = pd.read_csv(train_root+'ad.csv')
# click_log = pd.read_csv(train_root +'click_log.csv')
# user = pd.read_csv(train_root+'user.csv')
# Tclick_log = pd.read_csv(test_root +'click_log.csv')

In [4]:
pre_embedding_cre = np.load(train_root+'creative_id_w2v_300d_f.npy')
pre_vocab_cre = json.load(open(train_root+'creative_id_vocab_300d_f.json','r'))
pre_vocab_cre.update({'_PAD_':len(pre_vocab_cre)})
pre_embedding_cre = np.concatenate((pre_embedding_cre,np.zeros((1,300)) ))
print(len(pre_embedding_cre),len(pre_vocab_cre))

pre_embedding_uni = np.load(train_root+'uni_col_w2v_100d_f.npy')
pre_vocab_uni = json.load(open(train_root+'uni_col_vocab_100d_f.json','r'))
pre_vocab_uni.update({'_PAD_':len(pre_vocab_uni)})
pre_embedding_uni = np.concatenate((pre_embedding_uni,np.zeros((1,100)) ))
print(len(pre_embedding_uni),len(pre_vocab_uni))

vocab = {
    'creative':pre_vocab_cre,
    'uni_col':pre_vocab_uni,
}

4445721 4445721
110810 110810


In [5]:
%%time
train = pd.read_csv(train_root+'train_trf.csv',index_col = 0)
train

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


CPU times: user 31 s, sys: 1.97 s, total: 33 s
Wall time: 33 s


,user_id,creative_id,uni_col,click_times,time,age,gender
0,1,821396 877468 209778 1683713 122032 71691 1940...,7293_\N_326_5 29455_\N_106_5 9702_136_6_2 1466...,1 1 1 1 1 1 1 1 1 1 1 1 2,20 20 20 39 40 43 46 52 60 64 64 73 76,4,1
1,2,63441 155822 39714 609050 13069 441462 1266180...,22885_87_318_2 10686_80_238_2 18562_129_6_2 25...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,10 11 14 17 28 28 28 38 38 39 41 42 42 42 44 4...,10,1
2,3,661347 808612 593522 825434 710859 726940 3920...,32974_36256_\N_17 9877_40905_\N_17 17018_1674_...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,12 13 14 14 14 17 19 22 31 36 37 44 47 47 50 5...,7,2
3,4,39588 589886 574787 1892854 1230094 2264105 31...,19451_1862_238_2 7976_\N_25_18 13084_2625_248_...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,8 15 41 44 48 48 48 48 49 52 58 58 59 61 62 62...,5,1
4,5,296145 350759 24333 43235 852327 1054434 12964...,11882_\N_297_5 992_\N_\N_8 22885_87_318_2 9706...,1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,3 13 14 15 20 21 24 25 27 28 29 30 32 32 35 35...,4,1
...,...,...,...,...,...,...,...
2999995,2999996,190253 309607 1099 567833 571808 33159 1560316...,14681_\N_297_18 14681_\N_6_18 6753_26926_60_3 ...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,1 3 7 10 12 16 30 39 49 49 49 54 55 56 59 65 6...,4,2
2999996,2999997,20588 351431 15563 395549 169325 530723 43044 ...,11016_49_6_2 23057_1674_322_2 10988_1261_6_2 9...,1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 ...,3 3 5 5 6 8 10 11 14 18 19 19 20 21 22 26 27 2...,2,2
2999997,2999998,642652 932655 670 6508 944753 3507884 3123952 ...,29053_1567_6_2 274_\N_\N_16 8058_1334_317_2 16...,1 1 1 1 1 1 1 1 1 1 1,16 22 66 80 83 84 85 88 89 89 90,4,2
2999998,2999999,69204 602554 498385 69204 2293419 3007031 2996...,24952_26858_60_3 28581_35561_\N_4 28874_26858_...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1,7 9 19 32 63 65 67 69 75 75 75 75 76 76 79 79 ...,3,2


In [6]:
class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, uid, creative, uni_col,click_times, time, label=None):
        """Constructs a InputExample.
        Args:
            id: Unique id for the example.
            text: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
            label: (Optional) string. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.id = uid
        self.text = {
            'creative':creative,
            'uni_col':uni_col,
            'click_times':click_times,
            'time':time
        }
        self.label = label

class InputFeatures(object):
    def __init__(self,
                 example_id,
                 feature,
                 label
                 ):
        self.example_id = example_id
        creative, uni_col, click_times, time = feature
        self.feature = {
            'creative':creative,
            'uni_col':uni_col,
            'click_times':click_times,
            'time':time
        }
        self.label = label

def read_examples(df, is_training):
    if not is_training:
        df['label'] = np.zeros(len(df), dtype=np.int64)
    examples = []
    for idex, row in df.iterrows():
        if is_training:
            label = row['age']
        else:
            label = row['label']
        examples.append(InputExample(uid=idex, creative=row['creative_id'], uni_col =row['uni_col'], 
                                     click_times=row['click_times'], time = row['time'], label=label-1))
    return examples, df

def convert_examples_to_features(examples, word_to_id, max_seq_length):
    features = []
    for example_index, example in enumerate(tqdm(examples)):
        #!!!!!!!!!!!!!!!!!!!##
        content=['creative','uni_col']
        feature=[]
        for onekind in content:
            tokens_text = example.text[onekind].split()
            tokens=[]
            for token_text in tokens_text:
                if token_text in word_to_id[onekind]:
                    tokens.append(token_text)
                else:
                    tokens.append('_PAD_')
            if len(tokens)>=max_seq_length:
                tokens = tokens[:max_seq_length]
            else:
                tokens += ['_PAD_']*(max_seq_length-len(tokens))
            token_ids = [word_to_id[onekind][token] for token in tokens]
            feature.append(token_ids)
        ###for click_times
        tokens_text = example.text['click_times'].split()
        tokens=[]
        for token_text in tokens_text:
            if int(token_text)<=4:
                tokens.append(int(token_text))
            else:
                tokens.append(1)

        if len(tokens)>=max_seq_length:
            tokens = tokens[:max_seq_length]
        else:
            tokens += [0]*(max_seq_length-len(tokens))
        feature.append(tokens)
        ###for time
        tokens_text = example.text['time'].split()
        tokens=[]
        for token_text in tokens_text:
                tokens.append(int(token_text))

        if len(tokens)>=max_seq_length:
            tokens = tokens[:max_seq_length]
        else:
            tokens += [0]*(max_seq_length-len(tokens))
        feature.append(tokens)
            
        
        label = example.label

        if example_index < 1:
            print(example.id,'/n',feature,'/n',label)
#             logger.info("*** Example ***")
#             logger.info("idx: {}".format(example_index))
#             logger.info("id: {}".format(example.id))
#             logger.info("tokens: {}".format(' '.join(tokens).replace('\u2581', '_')))
#             logger.info("input_ids: {}".format(' '.join(map(str, input_ids))))
#             logger.info("input_mask: {}".format(len(input_mask)))
#             logger.info("segment_ids: {}".format(len(segment_ids)))
#             logger.info("label: {}".format(label))
        features.append(
            InputFeatures(
                example_id=example.id,
                feature= feature,
                label=label
            )
        )
    return features

def metric(y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='weighted')
    return acc, f1

In [7]:
import copy

class myModel(nn.Module):
    def __init__(self, hidden_size=768, num_class=10,dim=400,num_head=4,hidden=1024,dropout=0.1,max_seq_length=64):
        super(myModel, self).__init__()
        self.embedding_cre = nn.Embedding.from_pretrained(torch.tensor(pre_embedding_cre,dtype=torch.float32), freeze=True)
        self.embedding_uni = nn.Embedding.from_pretrained(torch.tensor(pre_embedding_uni,dtype=torch.float32), freeze=True)
        self.embeddingLN = nn.LayerNorm(dim, elementwise_affine=True)

        self.postion_embedding = Positional_Encoding(dim)
        self.encoder = Encoder(dim,num_head,hidden,dropout)#config.dim_model, config.num_head, config.hidden, config.dropout
        self.encoders = nn.ModuleList([
            copy.deepcopy(self.encoder)
            # Encoder(config.dim_model, config.num_head, config.hidden, config.dropout)
            for _ in range(1)])#config.num_encoder

        self.lstm = nn.LSTM(input_size=400, hidden_size=768, num_layers=2,
                            batch_first=True, bidirectional=True, dropout=0.5 )
#         self.fc = nn.Linear(hidden_size*2, num_class)

        # self.fc1 = nn.Linear(config.pad_size * config.dim_model, config.num_classes)
        # self.fc2 = nn.Linear(config.last_hidden, config.num_classes)
        # self.fc1 = nn.Linear(config.dim_model, config.num_classes)
        self.maxpool = nn.MaxPool1d(max_seq_length)
        self.fc = nn.Linear(hidden_size*2, num_class)


    def forward(self, cre, uni, clk, time):
        x_cre = self.embedding_cre(cre)
        x_uni = self.embedding_uni(uni)
        x = torch.cat((x_cre, x_uni), 2)
        x = self.embeddingLN(x)

        out_trf = self.postion_embedding(x, time)
        mask = clk.unsqueeze(1).expand(clk.size(0), clk.size(1), clk.size(1))#batch*seq*seq
        for encoder in self.encoders:
            out_trf = encoder(out_trf, mask)

        pad = torch.clamp(clk,0,1).unsqueeze(2)        
        out, _ = self.lstm(out_trf*pad)
#         out = self.fc(out[:,-1,:])
        # out = out.view(out.size(0), -1)#batch *seq*dim
        # # out = torch.mean(out, 1)
        # out = self.fc1(out)
#         out = torch.cat((out_trf, out), 2)
        out = out+(1-pad)*(-10000)
        out = F.tanh(out)
        out = out.permute(0, 2, 1)
        out = self.maxpool(out).squeeze()
        out = self.fc(out)

        return out


class Encoder(nn.Module):
    def __init__(self, dim_model, num_head, hidden, dropout):
        super(Encoder, self).__init__()
        self.attention = Multi_Head_Attention(dim_model, num_head, dropout)
        self.feed_forward = Position_wise_Feed_Forward(dim_model, hidden, dropout)

    def forward(self, x, mask):
        out = self.attention(x, mask)
        out = self.feed_forward(out)
        return out


class Positional_Encoding(nn.Module):
    def __init__(self, dim_model):
        super(Positional_Encoding, self).__init__()
        self.dim = dim_model
        self.layer_norm = nn.LayerNorm(dim_model)
#         self.dropout = nn.Dropout(0.1)
        pe = torch.tensor([[pos / (10000.0 ** (i // 2 * 2.0 / self.dim)) for i in range(self.dim)] for pos in range(92)],dtype=torch.float32)
        pe[:, 0::2] = np.sin(pe[:, 0::2])
        pe[:, 1::2] = np.cos(pe[:, 1::2])
        self.embedding_pos = nn.Embedding.from_pretrained(pe, freeze=True)
    def forward(self, x, time):

        x_pos = self.embedding_pos(time)
        out = x + x_pos
        out = self.layer_norm(out)
#         out = self.dropout(out)
        return out


class Scaled_Dot_Product_Attention(nn.Module):
    '''Scaled Dot-Product Attention '''
    def __init__(self):
        super(Scaled_Dot_Product_Attention, self).__init__()
#         self.atten_dropout = nn.Dropout(0.1)

    def forward(self, Q, K, V, scale=None, mask =None):

        attention = torch.matmul(Q, K.transpose(-1, -2))# scores : [batch_size x n_heads x len_q(=len_k) x len_k(=len_q)]
        if scale:
            attention = attention * scale
        if 1:  # TODO change this
            # attention = attention.masked_fill_(mask == 0, -1e9)
            attention = attention*torch.log2(1+mask)
        attention = F.softmax(attention, dim=-1)

        ###L dropout
#         attention = self.atten_dropout(attention)
        context = torch.matmul(attention, V)
        return context


class Multi_Head_Attention(nn.Module):
    def __init__(self, dim_model, num_head, dropout=0.1):
        super(Multi_Head_Attention, self).__init__()
        self.num_head = num_head
        assert dim_model % num_head == 0
        self.dim_head = dim_model // self.num_head
        self.fc_Q = nn.Linear(dim_model, num_head * self.dim_head)
        self.fc_K = nn.Linear(dim_model, num_head * self.dim_head)
        self.fc_V = nn.Linear(dim_model, num_head * self.dim_head)
        self.attention = Scaled_Dot_Product_Attention()
        self.fc = nn.Linear(num_head * self.dim_head, dim_model)
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(dim_model)

    def forward(self, x, mask):
        batch_size = x.size(0)
        Q = self.fc_Q(x)
        K = self.fc_K(x)
        V = self.fc_V(x)
        Q = Q.view(batch_size, -1, self.num_head, self.dim_head).transpose(1, 2)  # q_s: [batch_size x n_heads x len_q x d_k]
        K = K.view(batch_size, -1, self.num_head, self.dim_head).transpose(1, 2)
        V = V.view(batch_size, -1, self.num_head, self.dim_head).transpose(1, 2)

        if 1:  # TODO
            mask = mask.unsqueeze(1).repeat(1, self.num_head, 1, 1) # attn_mask : [batch_size x n_heads x len_q x len_k]  # TODO change this
        scale = K.size(-1) ** -0.5  # 缩放因子
        context = self.attention(Q, K, V, scale, mask)
        #context: [batch_size x len_q x n_heads * d_v]
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, self.dim_head * self.num_head)
        out = self.fc(context)
        out = self.dropout(out)
        out = out + x  # 残差连接
        out = self.layer_norm(out)
        return out


class Position_wise_Feed_Forward(nn.Module):
    def __init__(self, dim_model, hidden, dropout=0.0):
        super(Position_wise_Feed_Forward, self).__init__()
        self.fc1 = nn.Linear(dim_model, hidden)
        self.fc2 = nn.Linear(hidden, dim_model)
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(dim_model)

    def forward(self, x):
        out = self.fc1(x)
        out = F.gelu(out)
        out = self.fc2(out)
        out = self.dropout(out)
        out = out + x  # 残差连接
        out = self.layer_norm(out)
        return out

In [8]:
max_seq_length = 98
batch_size = 128
# tokenizer = jieba.lcut

learning_rate = 5e-4
num_epochs = 20
patience = 1

if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'
# device = 'cpu'
device = torch.device(device)

In [9]:
class SmoothLabelCritierion(nn.Module):
    """
    TODO:
    1. Add label smoothing
    2. Calculate loss
    """

    def __init__(self, label_smoothing=0.0):
        super(SmoothLabelCritierion, self).__init__()
        self.label_smoothing = label_smoothing
        self.LogSoftmax = nn.LogSoftmax()

        # When label smoothing is turned on, KL-divergence is minimized
        # If label smoothing value is set to zero, the loss
        # is equivalent to NLLLoss or CrossEntropyLoss.
        if label_smoothing > 0:
            self.criterion = nn.KLDivLoss(reduction='batchmean')
        else:
            self.criterion = nn.NLLLoss()
        self.confidence = 1.0 - label_smoothing

    def _smooth_label(self, num_tokens):

        one_hot = torch.randn(1, num_tokens)
        one_hot.fill_(self.label_smoothing / (num_tokens - 1))
        return one_hot

    def _bottle(self, v):
        return v.view(-1, v.size(2))

    def forward(self, dec_outs, labels):
        # Map the output to (0, 1)
        scores = self.LogSoftmax(dec_outs)
        # n_class
        num_tokens = scores.size(-1)

        gtruth = labels.view(-1)
        if self.confidence < 1:
            tdata = gtruth.detach()
            one_hot = self._smooth_label(num_tokens)
            if labels.is_cuda:
                one_hot = one_hot.cuda()
            tmp_ = one_hot.repeat(gtruth.size(0), 1)
            tmp_.scatter_(1, tdata.unsqueeze(1), self.confidence)
            gtruth = tmp_.detach()
        loss = self.criterion(scores, gtruth)
        return loss

In [9]:
train, valid = train_test_split(train, test_size=0.01, random_state=49, shuffle=True)
train.shape

(891000, 7)

In [10]:
%%time
train_examples, train_df = read_examples(train, is_training=True)
train_features = convert_examples_to_features(train_examples, vocab, max_seq_length)
valid_examples, valid_df = read_examples(valid, is_training=True)
valid_features = convert_examples_to_features(valid_examples, vocab, max_seq_length)

  0%|          | 1481/891000 [00:00<02:09, 6877.36it/s]

524247 /n [[10883, 323847, 57681, 60553, 1793252, 99604, 970091, 21, 64456, 4671, 6838, 289772, 94346, 49503, 1109620, 234878, 9394, 3907, 14099, 21, 520, 131272, 87310, 4061, 80, 1793253, 262276, 3907, 165971, 46591, 1793254, 323265, 1098558, 8301, 2705, 78787, 20, 133, 99531, 27, 250784, 497152, 85898, 20, 927773, 145926, 214463, 4276, 2449772, 20, 80, 27, 229239, 900892, 8621, 12535, 1166758, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772], [523, 2188, 3360, 7177, 2519, 662, 40876, 43, 1228, 966, 1490, 1638, 339, 11327, 662, 1764, 523, 1294, 554, 43, 0, 6235, 1297, 1294, 11, 2862, 13408, 1294, 346, 1294, 3184, 352, 10604, 16, 3790, 2580, 50, 11, 4692, 50, 2188, 343, 2

 20%|█▉        | 1789/9000 [00:00<00:00, 8904.35it/s]

222270 /n [[6, 7975, 82050, 19156, 28038, 17124, 1, 1651566, 26, 7628, 364318, 1, 251166, 26, 709, 408491, 9, 47214, 458369, 9, 74091, 20367, 6036, 1, 974, 1229, 1391, 36456, 42033, 15966, 5622, 25, 447, 284, 214507, 144974, 123107, 20482, 747908, 24055, 18553, 677147, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772, 3412772], [28, 1862, 1980, 10005, 548, 2729, 3, 31302, 43, 541, 6642, 3, 979, 43, 414, 452, 3, 4473, 3140, 3, 1890, 1109, 4671, 3, 811, 29, 32, 1316, 137, 348, 32, 8, 253, 388, 1515, 222, 2664, 4098, 11759, 2664, 476, 227, 98845, 98845

100%|██████████| 9000/9000 [00:01<00:00, 8941.35it/s]

CPU times: user 4min 56s, sys: 6.52 s, total: 5min 3s
Wall time: 5min 2s


In [11]:
train_input_cre = torch.tensor(np.array([train_feature.feature['creative'] for train_feature in tqdm(train_features)]), dtype=torch.long)
train_input_uni = torch.tensor(np.array([train_feature.feature['uni_col'] for train_feature in tqdm(train_features)]), dtype=torch.long)
train_input_clk= torch.tensor(np.array([train_feature.feature['click_times'] for train_feature in tqdm(train_features)]), dtype=torch.float32)
train_input_tim= torch.tensor(np.array([train_feature.feature['time'] for train_feature in tqdm(train_features)]), dtype=torch.long)
train_label = torch.tensor(np.array([train_feature.label for train_feature in train_features]), dtype=torch.long)

valid_input_cre = torch.tensor(np.array([valid_feature.feature['creative'] for valid_feature in valid_features]), dtype=torch.long)
valid_input_uni = torch.tensor(np.array([valid_feature.feature['uni_col'] for valid_feature in valid_features]), dtype=torch.long)
valid_input_clk= torch.tensor(np.array([valid_feature.feature['click_times'] for valid_feature in valid_features]), dtype=torch.float32)
valid_input_tim= torch.tensor(np.array([valid_feature.feature['time'] for valid_feature in valid_features]), dtype=torch.long)

valid_label = torch.tensor(np.array([valid_feature.label for valid_feature in valid_features]), dtype=torch.long)

train_data = torch.utils.data.TensorDataset(train_input_cre,train_input_uni,train_input_clk,train_input_tim, train_label)
valid_data = torch.utils.data.TensorDataset(valid_input_cre,valid_input_uni,valid_input_clk,valid_input_tim, valid_label)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size, shuffle=False)

100%|██████████| 891000/891000 [00:00<00:00, 1245067.91it/s]


In [13]:
torch.cuda.set_device(4)
model = myModel(768,10,max_seq_length=max_seq_length)
model.to(device)

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# scheduler_warm= torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda step:step/(2*len(train_loader)))

best_acc = 0.
valid_best = np.zeros((valid_label.size(0), 10))

early_stop = 0

In [61]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [9]:
###five fold
####

# train = pd.read_csv(train_root+'train_creative.csv',index_col = 0)

train_examples, train_df = read_examples(train, is_training=True)
train_features = convert_examples_to_features(train_examples, vocab, max_seq_length)

train_input_cre = torch.tensor(np.array([train_feature.feature['creative'] for train_feature in tqdm(train_features)]), dtype=torch.long)
train_input_uni = torch.tensor(np.array([train_feature.feature['uni_col'] for train_feature in tqdm(train_features)]), dtype=torch.long)
train_input_clk= torch.tensor(np.array([train_feature.feature['click_times'] for train_feature in tqdm(train_features)]), dtype=torch.float32)
train_input_tim= torch.tensor(np.array([train_feature.feature['time'] for train_feature in tqdm(train_features)]), dtype=torch.long)
train_label = torch.tensor(np.array([train_feature.label for train_feature in train_features]), dtype=torch.long)

import gc
del train
del train_df
del train_examples
del train_features

# del test_group
# del test_df 
# del test_examples
# del test_features
gc.collect()

  0%|          | 1759/3000000 [00:00<05:49, 8584.50it/s]

0 /n [[369522, 82667, 30791, 54327, 1122, 57, 17354, 2501, 3140, 1903, 18115, 1903, 150654, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720], [681, 3524, 2153, 2038, 1190, 42, 288, 2059, 880, 886, 847, 886, 17, 110809, 110809, 110809, 110809, 110809, 110809, 110809, 110809, 110809, 1

100%|██████████| 3000000/3000000 [00:02<00:00, 1337804.98it/s]


0

In [ ]:
###five fold
skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=107)
# off: out-of-fold
# oof_test = np.zeros((100_0000, 10), dtype=np.float32)
for fold, (train_index, valid_index) in enumerate(skf.split(train_label, train_label)):
    if fold!=1:
        continue

    print('================     fold {}        ==============='.format(fold))
    train_input_fold_cre = torch.tensor(train_input_cre[train_index], dtype=torch.long)
    train_input_fold_uni = torch.tensor(train_input_uni[train_index], dtype=torch.long)
    train_input_fold_clk = torch.tensor(train_input_clk[train_index], dtype=torch.float32)
    train_input_fold_tim = torch.tensor(train_input_tim[train_index], dtype=torch.long)
    train_label_fold = torch.tensor(train_label[train_index], dtype=torch.long)
    
    valid_input_fold_cre = torch.tensor(train_input_cre[valid_index], dtype=torch.long)
    valid_input_fold_uni = torch.tensor(train_input_uni[valid_index], dtype=torch.long)
    valid_input_fold_clk = torch.tensor(train_input_clk[valid_index], dtype=torch.float32)
    valid_input_fold_tim = torch.tensor(train_input_tim[valid_index], dtype=torch.long)
    valid_label_fold = torch.tensor(train_label[valid_index], dtype=torch.long)
    

    train_data_fold  = torch.utils.data.TensorDataset(train_input_fold_cre,train_input_fold_uni,train_input_fold_clk,train_input_fold_tim, train_label_fold )
    valid_data_fold  = torch.utils.data.TensorDataset(valid_input_fold_cre,valid_input_fold_uni,valid_input_fold_clk,valid_input_fold_tim,valid_label_fold )

    train_loader_fold  = torch.utils.data.DataLoader(train_data_fold , batch_size=batch_size, shuffle=True)
    valid_loader_fold  = torch.utils.data.DataLoader(valid_data_fold , batch_size=batch_size, shuffle=False)
    
     
        
    torch.cuda.set_device(3)
    model =  myModel(768,10,max_seq_length=max_seq_length)
    ###!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!####
    model.load_state_dict(torch.load(train_root+'lstm_age1.bin'))####TODO
    model.to(device)

    loss_fn = torch.nn.CrossEntropyLoss()
#     loss_fn = SmoothLabelCritierion(label_smoothing=0.1)
#     optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.035},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}]

    optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr=learning_rate, eps=1e-6)
    ###!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!####
    for param_group in optimizer.param_groups:####TODO
        param_group['lr']=1e-6

    best_val = 10000.
    valid_best = np.zeros((valid_label_fold.size(0), 10))

    early_stop = 0    
    model.train()
    
    del train_input_cre
    del train_input_uni
    del train_input_clk
    del train_input_tim
    del pre_embedding_cre
    del pre_embedding_uni
    gc.collect()
    for epoch in range(num_epochs):
#         if epoch==16:
#             model.load_state_dict(torch.load(train_root+'lstm_age1.bin'))
#             for param_group in optimizer.param_groups:
#                 param_group['lr']=1e-6
        model.train()
        train_loss = 0.
        for i, batch in enumerate(tqdm(train_loader_fold)):
            batch = tuple(t.to(device) for t in batch)
            x_cre,x_uni,x_clk,x_tim,  y_truth = batch
            y_pred = model( x_cre,x_uni,x_clk,x_tim)
            optimizer.zero_grad()

            loss = loss_fn(y_pred, y_truth)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() / len(train_loader_fold)
    # target.view(target.size(0), 1).long()
        model.eval()
        val_loss = 0.
        valid_preds_fold = np.zeros((valid_label_fold.size(0), 10))
        with torch.no_grad():
            for i, batch in tqdm(enumerate(valid_loader_fold)):
                batch = tuple(t.to(device) for t in batch)
                x_cre,x_uni,x_clk,x_tim,  y_truth = batch
                y_pred = model(x_cre,x_uni,x_clk,x_tim).detach()
                val_loss += loss_fn(y_pred, y_truth).item() / len(valid_loader_fold)
                valid_preds_fold[i * batch_size:(i + 1) * batch_size] = F.softmax(y_pred, dim=1).cpu().numpy()

        acc, f1 = metric(valid_label_fold, np.argmax(valid_preds_fold, axis=1))
        if best_val >= val_loss:
            early_stop = 0
            best_val = val_loss
            valid_best = valid_preds_fold
            # torch.save(model.state_dict(), 'model_fold_{}.bin'.format(fold))
            # # torch.save(model, os.path.join(MODEL_PATH, 'lmodel.pkl'))
            torch.save(model.state_dict(), train_root+'lstm_age1.bin')
        else:
            early_stop += 1
        print(
            'epoch: %d, train loss: %.8f, valid loss: %.8f, acc: %.8f, f1: %.8f, best_f1: %.8f\n' %
            (epoch, train_loss, val_loss, acc, f1, best_val))
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

        if early_stop >= patience:
            for param_group in optimizer.param_groups:
                if param_group['lr']>=3e-5:
                    param_group['lr'] *= 0.3
            for param_group in optimizer.param_groups:
                print(param_group['lr'])

            
    test_preds_fold = np.zeros((100_0000, 10))
    valid_preds_fold = np.zeros((valid_label_fold.size(0), 10))
    model.load_state_dict(torch.load(train_root+'lstm_age1.bin'))
    model.eval()
    
    with torch.no_grad():
        for i, batch in tqdm(enumerate(valid_loader_fold)):
            batch = tuple(t.to(device) for t in batch)
            x_cre,x_uni,x_clk,x_tim,  y_truth = batch
            y_pred = model(x_cre,x_uni,x_clk,x_tim).detach()
            val_loss += loss_fn(y_pred, y_truth).item() / len(valid_loader_fold)
            valid_preds_fold[i * batch_size:(i + 1) * batch_size] = F.softmax(y_pred, dim=1).cpu().numpy()
    with torch.no_grad():
        for i, batch in enumerate( tqdm(test_loader)):
            batch = tuple(t.to(device) for t in batch)
            x_cre,x_uni,x_clk,x_tim = batch
            y_pred = model(x_cre,x_uni,x_clk,x_tim).detach()
            test_preds_fold[i * batch_size:(i + 1) * batch_size] = F.softmax(y_pred, dim=1).cpu().numpy()
        
    acc, f1 = metric(valid_label_fold, np.argmax(valid_preds_fold, axis=1))
    print('epoch: best, acc: %.8f, f1: %.8f, best_f1: %.8f\n' %
                (acc, f1, best_val))
   
    
    #oof_test += test_preds_fold / 7 # uncomment this for 7 folds
#     oof_test += test_preds_fold / 5 # comment this line when training for 7 folds


    oof_test += test_preds_fold

================     fold 1        ===============


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: To 

epoch: 0, train loss: 1.08749237, valid loss: 1.17504477, acc: 0.51023200, f1: 0.50809725, best_f1: 1.17504477



100%|██████████| 17579/17579 [59:10<00:00,  4.95it/s] 
5860it [06:15, 15.60it/s]
  0%|          | 0/17579 [00:00<?, ?it/s]

epoch: 1, train loss: 1.08292990, valid loss: 1.17529610, acc: 0.51048267, f1: 0.50826154, best_f1: 1.17504477

1e-06
1e-06


100%|██████████| 17579/17579 [59:11<00:00,  4.95it/s] 
5860it [06:15, 15.62it/s]
  0%|          | 0/17579 [00:00<?, ?it/s]

epoch: 2, train loss: 1.08023783, valid loss: 1.17539755, acc: 0.51063467, f1: 0.50829112, best_f1: 1.17504477

1e-06
1e-06


 72%|███████▏  | 12685/17579 [42:42<16:24,  4.97it/s] 

In [14]:
torch.cuda.set_device(0)
test_preds_fold = np.zeros((100_0000, 10))
valid_preds_fold = np.zeros((valid_label_fold.size(0), 10))
model.load_state_dict(torch.load(train_root+'lstm_age1.bin', map_location={'cuda:3':'cuda:0'}))
model.eval()
# # model = myModel(768,10,max_seq_length=max_seq_length)
model.to(device)
with torch.no_grad():
    for i, batch in tqdm(enumerate(valid_loader_fold)):
        batch = tuple(t.to(device) for t in batch)
        x_cre,x_uni,x_clk,x_tim,  y_truth = batch
        y_pred = model(x_cre,x_uni,x_clk,x_tim).detach()
        val_loss += loss_fn(y_pred, y_truth).item() / len(valid_loader_fold)
        valid_preds_fold[i * batch_size:(i + 1) * batch_size] = F.softmax(y_pred, dim=1).cpu().numpy()
with torch.no_grad():
    for i, batch in enumerate( tqdm(test_loader)):
        batch = tuple(t.to(device) for t in batch)
        x_cre,x_uni,x_clk,x_tim = batch
        y_pred = model(x_cre,x_uni,x_clk,x_tim).detach()
        test_preds_fold[i * batch_size:(i + 1) * batch_size] = F.softmax(y_pred, dim=1).cpu().numpy()

acc, f1 = metric(valid_label_fold, np.argmax(valid_preds_fold, axis=1))
print('epoch: best, acc: %.8f, f1: %.8f, best_f1: %.8f\n' %
            (acc, f1, best_val))


#oof_test += test_preds_fold / 7 # uncomment this for 7 folds
#     oof_test += test_preds_fold / 5 # comment this line when training for 7 folds


# oof_test += test_preds_fold

0it [00:00, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
4688it [07:57,  9.83it/s]
100%|██████████| 7813/7813 [13:09<00:00,  9.89it/s]


epoch: best, acc: 0.51140000, f1: 0.50849440, best_f1: 0.91045324



In [16]:
class_num=10
weights = [1.0]*class_num

def search_weight(valid_y, raw_prob, init_weight=[1.0]*class_num, step=0.001):
    weight = init_weight.copy()
    f_best = accuracy_score(y_true=valid_y, y_pred=raw_prob.argmax(
        axis=1))
    flag_score = 0
    round_num = 1
    while(flag_score != f_best):
        print("round: ", round_num)
        round_num += 1
        flag_score = f_best
        for c in range(class_num):
            for n_w in range(0, 2000,10):
                num = n_w * step
                new_weight = weight.copy()
                new_weight[c] = num

                prob_df = raw_prob.copy()
                prob_df = prob_df * np.array(new_weight)

                f = accuracy_score(y_true=valid_y, y_pred=prob_df.argmax(
                    axis=1))
                if f > f_best:
                    weight = new_weight.copy()
                    f_best = f
                    print(f)
    return weight
search_weight(valid_label_fold,valid_preds_fold)

round:  1
0.511615
0.5116433333333333
0.511685
0.5117183333333334
0.51176
0.5117783333333333
0.511825
0.5118883333333333
0.5119
0.5119116666666667
0.5119733333333333
0.5120183333333334
0.5120316666666667
0.512045
0.51206
0.512075
0.512085
0.51209
0.5120916666666666
0.5121
round:  2
0.512105
0.5121066666666667
0.5121333333333333
0.5121483333333333
0.512165
round:  3
0.5121766666666666
0.5121833333333333
0.5121983333333333
0.5122066666666667
round:  4


[0.8, 0.99, 1.02, 1.02, 0.97, 1.0, 0.93, 0.97, 1.0, 1.12]

In [ ]:
oof_test = np.argmax(oof_test, axis=1)+1
oof_test
submission = pd.read_csv('submission2.csv', index_col = 0 )
submission.predicted_age = oof_test
submission
submission.to_csv('submission4.csv')

In [13]:
pd.DataFrame(test_preds_fold).to_csv('five51.csv')

In [15]:
acc

0.48612777777777777

In [14]:
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.
    for i, batch in enumerate(tqdm(train_loader)):
        batch = tuple(t.to(device) for t in batch)
        x_cre,x_uni,x_clk,x_tim, y_truth = batch
        y_pred = model( x_cre,x_uni,x_clk,x_tim)
        optimizer.zero_grad()
                                 
        loss = loss_fn(y_pred, y_truth)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() / len(train_loader)
#         if epoch<2:
#             scheduler_warm.step()
# target.view(target.size(0), 1).long()
    model.eval()
    val_loss = 0.
    valid_preds_fold = np.zeros((valid_label.size(0), 10))
    with torch.no_grad():
        for i, batch in tqdm(enumerate(valid_loader)):
            batch = tuple(t.to(device) for t in batch)
            x_cre,x_uni,x_clk,x_tim,  y_truth = batch
            y_pred = model(x_cre,x_uni,x_clk,x_tim).detach()
            val_loss += loss_fn(y_pred, y_truth).item() / len(valid_loader)
            valid_preds_fold[i * batch_size:(i + 1) * batch_size] = F.softmax(y_pred, dim=1).cpu().numpy()

    acc, f1 = metric(valid_label, np.argmax(valid_preds_fold, axis=1))
    if best_acc <= acc:
        early_stop = 0
        best_acc = acc
        valid_best = valid_preds_fold
        # torch.save(model.state_dict(), 'model_fold_{}.bin'.format(fold))
        # # torch.save(model, os.path.join(MODEL_PATH, 'lmodel.pkl'))
        torch.save(model.state_dict(), train_root+'lstm_age_trf9.bin')
    else:
        early_stop += 1
    print(
        'epoch: %d, train loss: %.8f, valid loss: %.8f, acc: %.8f, f1: %.8f, best_acc: %.8f\n' %
        (epoch, train_loss, val_loss, acc, f1, best_acc))
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    if early_stop >= patience:
#         break
        for param_group in optimizer.param_groups:
            if param_group['lr']>=3e-5:
                param_group['lr'] *= 0.3
        for param_group in optimizer.param_groups:
            print(param_group['lr'])
#     for param_group in optimizer.param_groups:
#         print(param_group['lr'])


  0%|          | 0/6961 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
  0%|          | 1/6961 [00:00<1:04:08,  1.81it/s]

tensor([[[    -0.],
         [    -0.],
         [    -0.],
         ...,
         [-10000.],
         [-10000.],
         [-10000.]],

        [[    -0.],
         [    -0.],
         [    -0.],
         ...,
         [-10000.],
         [-10000.],
         [-10000.]],

        [[    -0.],
         [    -0.],
         [    -0.],
         ...,
         [-10000.],
         [-10000.],
         [-10000.]],

        ...,

        [[    -0.],
         [    -0.],
         [    -0.],
         ...,
         [-10000.],
         [-10000.],
         [-10000.]],

        [[    -0.],
         [    -0.],
         [    -0.],
         ...,
         [-10000.],
         [-10000.],
         [-10000.]],

        [[    -0.],
         [    -0.],
         [    -0.],
         ...,
         [-10000.],
         [-10000.],
         [-10000.]]], device='cuda:4') tensor([[[-3.9947e-02, -6.1290e-02, -2.1701e-02,  ..., -7.0041e-03,
           2.6523e-02, -1.7402e-02],
         [-3.4824e-03, -7.1104e-02, -4.5632e-02,

  0%|          | 2/6961 [00:00<52:42,  2.20it/s]  

tensor([[[    -0.],
         [    -0.],
         [    -0.],
         ...,
         [-10000.],
         [-10000.],
         [-10000.]],

        [[    -0.],
         [    -0.],
         [    -0.],
         ...,
         [-10000.],
         [-10000.],
         [-10000.]],

        [[    -0.],
         [    -0.],
         [    -0.],
         ...,
         [    -0.],
         [    -0.],
         [    -0.]],

        ...,

        [[    -0.],
         [    -0.],
         [    -0.],
         ...,
         [-10000.],
         [-10000.],
         [-10000.]],

        [[    -0.],
         [    -0.],
         [    -0.],
         ...,
         [-10000.],
         [-10000.],
         [-10000.]],

        [[    -0.],
         [    -0.],
         [    -0.],
         ...,
         [-10000.],
         [-10000.],
         [-10000.]]], device='cuda:4') tensor([[[-2.4654e-02, -4.5801e-02,  5.4477e-02,  ..., -2.3522e-01,
          -4.3820e-02,  8.7354e-02],
         [ 2.2563e-02, -6.6790e-02,  1.0983e-02,

  0%|          | 3/6961 [00:01<45:35,  2.54it/s]

...

  0%|          | 12/6961 [00:03<26:40,  4.34it/s]

tensor([[[    -0.],
         [    -0.],
         [    -0.],
         ...,
         [    -0.],
         [    -0.],
         [    -0.]],

        [[    -0.],
         [    -0.],
         [    -0.],
         ...,
         [-10000.],
         [-10000.],
         [-10000.]],

        [[    -0.],
         [    -0.],
         [    -0.],
         ...,
         [-10000.],
         [-10000.],
         [-10000.]],

        ...,

        [[    -0.],
         [    -0.],
         [    -0.],
         ...,
         [-10000.],
         [-10000.],
         [-10000.]],

        [[    -0.],
         [    -0.],
         [    -0.],
         ...,
         [-10000.],
         [-10000.],
         [-10000.]],

        [[    -0.],
         [    -0.],
         [    -0.],
         ...,
         [-10000.],
         [-10000.],
         [-10000.]]], device='cuda:4') tensor([[[ 3.4095e-02, -5.3203e-02, -3.0383e-02,  ..., -2.2888e-01,
          -9.1916e-02,  7.0748e-02],
         [ 4.2636e-02, -1.1759e-01, -5.6108e-02,

  0%|          | 12/6961 [00:03<32:11,  3.60it/s]

tensor([[[    -0.],
         [    -0.],
         [    -0.],
         ...,
         [-10000.],
         [-10000.],
         [-10000.]],

        [[    -0.],
         [    -0.],
         [    -0.],
         ...,
         [-10000.],
         [-10000.],
         [-10000.]],

        [[    -0.],
         [    -0.],
         [    -0.],
         ...,
         [-10000.],
         [-10000.],
         [-10000.]],

        ...,

        [[    -0.],
         [    -0.],
         [    -0.],
         ...,
         [-10000.],
         [-10000.],
         [-10000.]],

        [[    -0.],
         [    -0.],
         [    -0.],
         ...,
         [-10000.],
         [-10000.],
         [-10000.]],

        [[    -0.],
         [    -0.],
         [    -0.],
         ...,
         [-10000.],
         [-10000.],
         [-10000.]]], device='cuda:4') tensor([[[ 2.5090e-02, -2.7420e-02, -2.3216e-02,  ..., -2.4549e-01,
          -7.0854e-02,  6.7338e-02],
         [-2.4378e-02, -1.1291e-01, -2.1170e-02,

KeyboardInterrupt: 

In [17]:
for param_group in optimizer.param_groups:#copy4 0.4862,0.4884
        param_group['lr']=1e-6

In [ ]:
2020.5.12
0.39 0.92
V1    age n_vocab不冻结0.39√    gender n_vocab 冻结 
n_vocab在目前情况下看来不是效果很好
V2    age all冻结 0.4528   gender all冻结0.9432
V3    age cre与adv  lr-3 0.46 0.470 lr-4 0.4760   
V4    age cre-adv-ind lr-4 0.4768

In [9]:
test_group = pd.read_csv(test_root+'test_trf.csv',index_col=0)
test_group

,user_id,creative_id,uni_col,click_times,time
0,3000001,351878 103064 665090 593698 1508864 1797787 17...,7579_\N_322_18 13084_1794_248_2 7000_1701_247_...,1 1 1 1 1 1 1 1 1 1 1,3 11 11 23 29 49 49 49 54 54 82
1,3000002,152519 151984 176984 12838 72773 64667 81234 7...,12993_1674_322_2 27800_\N_24_18 24661_1687_5_2...,1 2 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 ...,9 10 12 12 15 17 23 23 24 25 26 26 30 30 31 31...
2,3000003,161840 367084 36634 115761 73137 150407 41212 ...,22367_82_319_2 40491_34504_202_5 22338_2065_23...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,1 3 3 3 3 4 4 11 11 15 19 21 27 37 38 43 55 58...
3,3000004,108656 849706 678427 9870 157180 94025 907546 ...,8520_1036_319_2 28323_\N_40_18 2286_\N_54_18 8...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1,13 15 16 18 18 23 27 32 35 40 40 40 40 62 63 6...
4,3000005,123860 183003 40625 26793 71219 259607 167448 ...,30710_1896_238_2 29243_1987_26_2 23746_129_6_2...,1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,2 3 6 7 7 8 9 13 26 27 28 33 35 38 51 55 62 65...
...,...,...,...,...,...
999995,3999996,172004 237425 54100 417864 282643 30856 116356...,10985_1261_6_2 7725_1404_2_2 10987_1261_6_2 35...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 1 1 1 1 1 2 ...,1 1 2 5 8 10 12 24 28 28 30 34 35 35 40 40 40 ...
999996,3999997,271727 390953 914417 641757 50445 14993 793940...,10986_\N_6_12 16639_1246_81_2 15102_\N_222_5 3...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1,2 10 17 18 18 20 30 30 45 47 48 57 59 65 67 82...
999997,3999998,100456 464055 54150 316582 882346 882346 77124...,23365_\N_27_18 28073_2344_167_2 10989_\N_202_1...,1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1,4 9 12 15 17 22 25 32 39 46 51 61 65 68 68 75 ...
999998,3999999,273715 95215 146101 117043 840 366940 148209 7...,37660_\N_322_18 22626_1062_238_2 20528_1492_32...,1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 ...,2 2 2 3 6 7 8 12 19 20 21 26 26 26 26 29 30 31...


In [10]:
%%time
test_examples, test_df = read_examples(test_group, is_training=False)
test_features = convert_examples_to_features(test_examples, vocab, max_seq_length)

test_input_cre = torch.tensor(np.array([test_feature.feature['creative']  for test_feature in tqdm(test_features)]), dtype=torch.long)
test_input_uni = torch.tensor(np.array([test_feature.feature['uni_col']  for test_feature in tqdm(test_features)]), dtype=torch.long)
test_input_clk = torch.tensor(np.array([test_feature.feature['click_times']  for test_feature in tqdm(test_features)]), dtype=torch.float32)
test_input_tim = torch.tensor(np.array([test_feature.feature['time']  for test_feature in tqdm(test_features)]), dtype=torch.long)


test_data = torch.utils.data.TensorDataset(test_input_cre, test_input_uni,test_input_clk,test_input_tim)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False)

  0%|          | 1259/1000000 [00:00<03:00, 5545.06it/s]

0 /n [[111563, 1182, 46816, 21269, 2050, 1834, 1289, 1971, 102895, 37550, 4012, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720, 4445720], [605, 185, 490, 8986, 711, 711, 158, 342, 6232, 6279, 868, 110809, 110809, 110809, 110809, 110809, 110809, 110809, 110809, 110809, 11080

100%|██████████| 1000000/1000000 [00:00<00:00, 1286721.04it/s]


CPU times: user 6min 47s, sys: 13.9 s, total: 7min 1s
Wall time: 6min 54s


In [11]:
torch.cuda.set_device(3)
model = myModel(768,10,max_seq_length=max_seq_length)
model.load_state_dict(torch.load(train_root+'lstm_age1.bin'))
model.eval()
model.to(device)

myModel(
  (embedding_cre): Embedding(4445721, 300)
  (embedding_uni): Embedding(110810, 100)
  (embeddingLN): LayerNorm((400,), eps=1e-05, elementwise_affine=True)
  (postion_embedding): Positional_Encoding(
    (layer_norm): LayerNorm((400,), eps=1e-05, elementwise_affine=True)
    (embedding_pos): Embedding(92, 400)
  )
  (encoder): Encoder(
    (attention): Multi_Head_Attention(
      (fc_Q): Linear(in_features=400, out_features=400, bias=True)
      (fc_K): Linear(in_features=400, out_features=400, bias=True)
      (fc_V): Linear(in_features=400, out_features=400, bias=True)
      (attention): Scaled_Dot_Product_Attention()
      (fc): Linear(in_features=400, out_features=400, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (layer_norm): LayerNorm((400,), eps=1e-05, elementwise_affine=True)
    )
    (feed_forward): Position_wise_Feed_Forward(
      (fc1): Linear(in_features=400, out_features=1024, bias=True)
      (fc2): Linear(in_features=1024, out_features=400, 

In [12]:
test_preds_fold = np.zeros((len(test_df), 10))
with torch.no_grad():
    for i, batch in enumerate( tqdm(test_loader)):
        batch = tuple(t.to(device) for t in batch)
        x_cre, x_uni, x_clk,x_tim= batch
        y_pred = model(x_cre, x_uni, x_clk,x_tim).detach()
        test_preds_fold[i * batch_size:(i + 1) * batch_size] = F.softmax(y_pred, dim=1).cpu().numpy()
        
# test_preds_fold = np.argmax(test_preds_fold, axis=1)+1
test_preds_fold

  0%|          | 0/7813 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
100%|██████████| 7813/7813 [08:16<00:00, 15.73it/s]


array([[1.90295260e-02, 5.87537408e-01, 3.39540720e-01, ...,
        1.10562733e-05, 5.03552155e-06, 1.02033812e-06],
       [4.61107629e-05, 3.16065439e-06, 2.79334336e-06, ...,
        2.14797795e-01, 1.85888421e-04, 2.66915572e-06],
       [2.20072875e-03, 1.89610466e-01, 2.34682947e-01, ...,
        5.97620674e-04, 6.77099961e-05, 4.23706988e-05],
       ...,
       [1.50729358e-01, 7.54850090e-01, 9.39997584e-02, ...,
        7.05929040e-07, 4.42586469e-07, 2.23300944e-07],
       [3.68597481e-04, 3.28052223e-01, 6.70274436e-01, ...,
        1.45324478e-07, 8.01207491e-08, 1.86131359e-08],
       [1.08797918e-04, 2.18730792e-03, 1.01299472e-02, ...,
        5.63489084e-05, 1.33069682e-06, 6.83924611e-07]])

In [3]:
submission = pd.read_csv('submission2.csv', index_col = 0 )
submission.predicted_age = test_preds_fold
submission

,predicted_age,predicted_gender
user_id,,
3000001,3,0
3000002,7,0
3000003,2,0
3000004,3,0
3000005,4,0
...,...,...
3999996,3,0
3999997,2,0
3999998,2,0


In [4]:
submission.to_csv('submission4.csv')

In [13]:
torch.log2(a+1)

tensor([0.])

In [1]:
a=torch.tensor([0],dtype=torch.float)

NameError: name 'torch' is not defined